<a href="https://colab.research.google.com/github/KurniaKhaikal/Cifar10k_TransferLearning/blob/main/Feature2_Cifar10_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
from glob import glob
import random
import matplotlib.pylab as plt

In [3]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"khaikal","key":"9e492a7553b8e713debda6f4f6810151"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d oxcdcd/cifar10

 99% 174M/175M [00:01<00:00, 161MB/s]
100% 175M/175M [00:01<00:00, 175MB/s]


In [5]:
import os
import zipfile

local_zip = '/content/cifar10.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [6]:
!mkdir "/content/cifar10/Base2"

In [7]:
base_dataset = "/content/cifar10/Base2"

In [8]:
class_dir = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
IMAGE_SIZE = (224, 224)

In [9]:
!pip install split-folders

In [10]:
import splitfolders


splitfolders.ratio("/content/cifar10/test", output="/content/cifar10/Base2",
    seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

Copying files: 10000 files [00:01, 7060.30 files/s]


In [11]:
import os
# indexing file images
dataset_train = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'train', class_item)
    for file in os.listdir(cur_dir):
        dataset_train.append(os.path.join(cur_dir, file))

In [12]:
print("train:", len(dataset_train))

train: 9000


In [13]:
from imutils import paths
import cv2
from sklearn.preprocessing import LabelBinarizer

In [14]:
print("[INFO] load images Cifar10k dataset...")
#  load images
train_images = []
for image_path in dataset_train:
    if ".jpg" or ".jpeg" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        train_images.append(image)

[INFO] load images Cifar10k dataset...


In [15]:
import numpy as np
# normalization
print("[INFO] normalization...")
train_x = np.array(train_images).astype("float32") / 255.0

[INFO] normalization...


In [16]:
import numpy as np
import json
import os
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model

In [17]:
from tensorflow.keras.applications.vgg19 import VGG19
modelVGG19 = VGG19(weights="imagenet")
model = Model(inputs=modelVGG19.input, outputs=modelVGG19.get_layer("fc1").output)
model.summary()

574726144/574710816 [==============================] - 5s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [18]:
features = model.predict(train_x)

In [19]:
indexes = list(range(0, train_x.shape[0]))
features_array = [[float(x) for x in y] for y in features]
labels = [path.split("/")[5] for path in dataset_train]
data_train = {"indexes": indexes, "features": features_array, "locations": dataset_train, "labels":labels}

In [20]:
with open('/content/feature_extraction.json', 'w') as f:
    json.dump(data_train, f)